Spark ile basit bir sekilde dataFrame olusturup sql sorgulariyla gosterme

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col 
from pyspark.sql.types import StringType, BooleanType, DateType
from pyspark.sql.functions import when
from pyspark.sql.types import StructType,StructField, StringType, IntegerType , FloatType

spark = SparkSession.builder.appName("DataFrame example 3").getOrCreate()
columns = ["firstname", "age", "jobStartDate", "isGraduated", "gender", "salary"]
data = [("James", 34, "2006-01-01", "true", "M", 3000.60), ("Michael", 33, "1980-01-10", "true", "F", 3300.80), ("Robert", 37, "06-01-1992", "false", "M", 5000.50)]

df = spark.createDataFrame(data=data, schema=columns)

df2 = df.withColumn("age", col("age").cast(StringType())).withColumn("isGraduated", col("isGraduated").cast(BooleanType())).withColumn("jobStartDate", col("jobStartDate").cast(DateType()))

df3 = df2.selectExpr("cast(age as int) age", "cast(isGraduated as string) isGraduated", "cast(jobStartDate as string) jobStartDate")
df3.createOrReplaceTempView("my_temp_view")

spark.sql("SELECT * FROM my_temp_view").show()

23/04/12 20:59:23 WARN Utils: Your hostname, icecream resolves to a loopback address: 127.0.1.1; using 192.168.1.105 instead (on interface wlo1)
23/04/12 20:59:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/12 20:59:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+-----------+------------+
|age|isGraduated|jobStartDate|
+---+-----------+------------+
| 34|       true|  2006-01-01|
| 33|       true|  1980-01-10|
| 37|      false|        null|
+---+-----------+------------+



In [12]:
# dataframe'deki sütunların veri tiplerini değiştirme işlemi
df2 = df.withColumn("age", col("age").cast(StringType())) \
        .withColumn("isGraduated", col("isGraduated").cast(BooleanType())) \
        .withColumn("jobStartDate", col("jobStartDate").cast(DateType()))

# dataframe'in şemasını yazdırma
df2.printSchema()
print("Code is Here : 1")

# sütunları seçme ve veri tiplerini değiştirme işlemi
df3 = df2.selectExpr("cast(age as int) age", "cast(isGraduated as string) isGraduated", "cast(jobStartDate as string) jobStartDate")

# dataframe'in şemasını yazdırma
df3.printSchema()

# dataframe'i gösterme
df3.show(truncate=False)

# dataframe'i temporary view olarak kaydetme
df3.createOrReplaceTempView("CastExample")

# SQL sorgusu ile temporary view'den yeni bir dataframe oluşturma ve veri tiplerini değiştirme işlemi
df4 = spark.sql("SELECT STRING(age), BOOLEAN(isGraduated), DATE(jobStartDate) FROM CastExample")

# dataframe'in şemasını yazdırma
df4.printSchema()

# dataframe'i gösterme
df4.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- jobStartDate: date (nullable = true)
 |-- isGraduated: boolean (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)

Code is Here : 1
root
 |-- age: integer (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- jobStartDate: string (nullable = true)

+---+-----------+------------+
|age|isGraduated|jobStartDate|
+---+-----------+------------+
|34 |true       |2006-01-01  |
|33 |true       |1980-01-10  |
|37 |false      |null        |
+---+-----------+------------+

root
 |-- age: string (nullable = true)
 |-- isGraduated: boolean (nullable = true)
 |-- jobStartDate: date (nullable = true)

+---+-----------+------------+
|age|isGraduated|jobStartDate|
+---+-----------+------------+
|34 |true       |2006-01-01  |
|33 |true       |1980-01-10  |
|37 |false      |null        |
+---+-----------+------------+



F = Female ve M = Male olarak bu ikisine uymayanlarida NA olarak Gosterme ve Maaslarini 3 katina cikarma

In [30]:
from pyspark.sql.functions import when

newData = [('James', 'Smith', 'M', 3000),
        ('Anna', 'Rose', 'F', 4100),
        ('Robert', 'Williams', 'NA', 6200),
        (None, 'Rob', 'F', 6200)]

newColumns = ["first_name", "last_name", "gender", "salary"]

myDf = spark.createDataFrame(data=newData, schema=newColumns)

myDf = myDf.withColumn("genderType", when(myDf["gender"] == "M", "male")
.when(myDf["gender"] == "F", "female")
.otherwise("NA"))

myDf = myDf.withColumn("newSalary", myDf["salary"] * 3)

myDf.show()


+----------+---------+------+------+----------+---------+
|first_name|last_name|gender|salary|genderType|newSalary|
+----------+---------+------+------+----------+---------+
|     James|    Smith|     M|  3000|      male|     9000|
|      Anna|     Rose|     F|  4100|    female|    12300|
|    Robert| Williams|    NA|  6200|        NA|    18600|
|      null|      Rob|     F|  6200|    female|    18600|
+----------+---------+------+------+----------+---------+



yas 20 den buyuk erkek ve abc universitesine giden kisileri gosterme

In [15]:
schoolData = [("Ali", "Yılmaz", "ABC Üniversitesi", 22, "Erkek"), ("Ayşe", "Kara", "DEF Üniversitesi", 19, "Kadın"), ("Mehmet", "Öztürk", "ABC Üniversitesi", 21, "Erkek"), ("Zeynep", "Aksoy", "GHI Üniversitesi", 20, "Kadın"), ("Ahmet", "Demir", "ABC Üniversitesi", 23, "Erkek")]
schoolSchema = StructType([ StructField("Ad", StringType(), True), StructField("Soyad", StringType(), True), StructField("Okul", StringType(), True), StructField("Yas", IntegerType(), True), StructField("Cinsiyet", StringType(), True) ])

schoolDf = spark.createDataFrame(data=schoolData, schema=schoolSchema)

selectedData = schoolDf.filter((col('Yas') > 20) & (col('Cinsiyet') == "Erkek") & (col("Okul") == "ABC Üniversitesi")).select("*")

selectedData.show()
selectedData.count()

+------+------+----------------+---+--------+
|    Ad| Soyad|            Okul|Yas|Cinsiyet|
+------+------+----------------+---+--------+
|   Ali|Yılmaz|ABC Üniversitesi| 22|   Erkek|
|Mehmet|Öztürk|ABC Üniversitesi| 21|   Erkek|
| Ahmet| Demir|ABC Üniversitesi| 23|   Erkek|
+------+------+----------------+---+--------+



3

yas 20 - 25 arasi , ABC universitesi , erkek veya kadin , not > 80 den buyuk olucak sekilde olanlari gosteriyor

In [28]:
schoolData = [("Ali", "Yılmaz", "ABC Üniversitesi", 22, "Erkek", 85.5), ("Ayşe", "Kara", "DEF Üniversitesi", 19, "Kadın", 78.0), ("Mehmet", "Öztürk", "ABC Üniversitesi", 21, "Erkek", 89.5), ("Zeynep", "Aksoy", "GHI Üniversitesi", 20, "Kadın", 91.0), ("Ahmet", "Demir", "ABC Üniversitesi", 23, "Erkek", 76.5)]
schoolSchema = StructType([ StructField("Ad", StringType(), True), StructField("Soyad", StringType(), True), StructField("Okul", StringType(), True), StructField("Yas", IntegerType(), True), StructField("Cinsiyet", StringType(), True), StructField("Not", FloatType(), True) ])

schoolDf = spark.createDataFrame(data=schoolData , schema=schoolSchema)

schoolDf.show()

selectedData = schoolDf.filter((col("Okul") == "ABC Üniversitesi") & ((col("Cinsiyet") == "Erkek") | (col("Cinsiyet") == "Kadın")) & (col("Yas").between(20, 25)) & (col("Not") > 80)).select("*")

selectedData.show()

+------+------+----------------+---+--------+----+
|    Ad| Soyad|            Okul|Yas|Cinsiyet| Not|
+------+------+----------------+---+--------+----+
|   Ali|Yılmaz|ABC Üniversitesi| 22|   Erkek|85.5|
|  Ayşe|  Kara|DEF Üniversitesi| 19|   Kadın|78.0|
|Mehmet|Öztürk|ABC Üniversitesi| 21|   Erkek|89.5|
|Zeynep| Aksoy|GHI Üniversitesi| 20|   Kadın|91.0|
| Ahmet| Demir|ABC Üniversitesi| 23|   Erkek|76.5|
+------+------+----------------+---+--------+----+

+------+------+----------------+---+--------+----+
|    Ad| Soyad|            Okul|Yas|Cinsiyet| Not|
+------+------+----------------+---+--------+----+
|   Ali|Yılmaz|ABC Üniversitesi| 22|   Erkek|85.5|
|Mehmet|Öztürk|ABC Üniversitesi| 21|   Erkek|89.5|
+------+------+----------------+---+--------+----+

